In [33]:
import numpy as np
from collections import defaultdict

class SparseMultiHistogram:
    def __init__(self, bin_edges):
        """
        bin_edges: list of 1D numpy arrays for each dimension (same as np.histogramdd)
        """
        self.bin_edges = [np.asarray(e) for e in bin_edges]
        self.ndim = len(bin_edges)
        self.hist = defaultdict(float)
        self.total_count = 0.0
        self._cdf_cache = None  # for sampling

    def _digitize(self, samples):
        """Convert continuous samples to bin indices, matching np.histogramdd behavior."""
        indices = [
            np.searchsorted(self.bin_edges[d], samples[:, d], side='right') - 1
            for d in range(self.ndim)
        ]
        # Clip to ensure valid bin indices
        for d in range(self.ndim):
            indices[d] = np.clip(indices[d], 0, len(self.bin_edges[d]) - 2)
        return np.stack(indices, axis=1)

    def add_samples(self, samples, weight=1.0):
        """Accumulate histogram counts for samples (N, D)."""
        indices = self._digitize(samples)
        for idx in map(tuple, indices):
            self.hist[idx] += weight
        self.total_count += len(samples) * weight
        self._cdf_cache = None  # invalidate cache

    def normalize(self):
        """Normalize histogram to form a probability distribution."""
        if self.total_count > 0:
            for k in self.hist:
                self.hist[k] /= self.total_count
            self.total_count = 1.0
        self._cdf_cache = None

    def sample(self, n_samples):
        """Draw samples according to histogram probability."""
        if self._cdf_cache is None:
            keys = np.array(list(self.hist.keys()))
            probs = np.array(list(self.hist.values()), dtype=float)
            probs /= probs.sum()
            self._cdf_cache = (keys, probs)

        keys, probs = self._cdf_cache
        chosen_idx = np.random.choice(len(probs), size=n_samples, p=probs)
        chosen_bins = keys[chosen_idx]

        # Uniform random inside each bin
        lows = np.array([self.bin_edges[d][chosen_bins[:, d]] for d in range(self.ndim)]).T
        highs = np.array([self.bin_edges[d][chosen_bins[:, d] + 1] for d in range(self.ndim)]).T
        r = np.random.rand(n_samples, self.ndim)
        return lows + r * (highs - lows)

In [40]:
bin_edges = [np.linspace(0, 1, 33)] * 3
hist = SparseMultiHistogram(bin_edges)

# 加入樣本
samples = np.random.rand(10000, 3)
hist.add_samples(samples)

# 正規化成機率分布
hist.normalize()

# 取樣
new_samples = hist.sample(1000)

print("Unique bins stored:", len(hist.hist))
print("Sample shape:", new_samples.shape)
print(hist.hist)

Unique bins stored: 8614
Sample shape: (1000, 3)
defaultdict(<class 'float'>, {(np.int64(7), np.int64(24), np.int64(28)): 0.0001, (np.int64(3), np.int64(4), np.int64(13)): 0.0004, (np.int64(16), np.int64(24), np.int64(15)): 0.0001, (np.int64(6), np.int64(30), np.int64(5)): 0.0002, (np.int64(31), np.int64(14), np.int64(8)): 0.0002, (np.int64(24), np.int64(21), np.int64(14)): 0.0001, (np.int64(7), np.int64(13), np.int64(16)): 0.0001, (np.int64(28), np.int64(17), np.int64(10)): 0.0001, (np.int64(17), np.int64(13), np.int64(18)): 0.0001, (np.int64(30), np.int64(19), np.int64(25)): 0.0003, (np.int64(7), np.int64(6), np.int64(16)): 0.0002, (np.int64(9), np.int64(28), np.int64(14)): 0.0003, (np.int64(17), np.int64(3), np.int64(8)): 0.0003, (np.int64(30), np.int64(21), np.int64(2)): 0.0002, (np.int64(28), np.int64(3), np.int64(12)): 0.0001, (np.int64(22), np.int64(24), np.int64(3)): 0.0001, (np.int64(16), np.int64(1), np.int64(10)): 0.0002, (np.int64(18), np.int64(26), np.int64(2)): 0.0001, (n

In [35]:
import numpy as np
from collections import defaultdict

# 測試資料
samples = np.random.rand(100000, 3)
bin_edges = [np.linspace(0, 1, 9)] * 3

# 你的 SparseMultiHistogram
h = SparseMultiHistogram(bin_edges)
h.add_samples(samples)
h.normalize()

# numpy 版本
ref_hist, _ = np.histogramdd(samples, bins=bin_edges)
ref_hist = ref_hist / ref_hist.sum()

# flatten
ref_flat = ref_hist.ravel()
sparse_flat = np.zeros_like(ref_flat)
shape = ref_hist.shape
for k, v in h.hist.items():
    sparse_flat[np.ravel_multi_index(k, shape)] = v

rmse = np.sqrt(np.mean((ref_flat - sparse_flat) ** 2))
print("RMSE =", rmse)

RMSE = 0.0


In [36]:
def rmse(hist1, hist2):
    """
    Compute RMSE between two SparseMultiHistogram objects.
    Both must have same bin_edges and dimension.
    """
    assert hist1.ndim == hist2.ndim, "Dim mismatch"
    for e1, e2 in zip(hist1.bin_edges, hist2.bin_edges):
        if not np.allclose(e1, e2):
            raise ValueError("Bin edges mismatch between histograms")

    # 所有出現過的 bin（聯集）
    all_keys = set(hist1.hist.keys()) | set(hist2.hist.keys())

    # 計算平方差
    sq_err = 0.0
    for key in all_keys:
        v1 = hist1.hist.get(key, 0.0)
        v2 = hist2.hist.get(key, 0.0)
        diff = v1 - v2
        sq_err += diff * diff

    # RMSE = sqrt(mean(square error))
    rmse_value = np.sqrt(sq_err / len(all_keys))
    return rmse_value


bin_edges = [np.linspace(0, 1, 33)] * 3
h1 = SparseMultiHistogram(bin_edges)
h2 = SparseMultiHistogram(bin_edges)

data1=np.random.rand(10000, 3)
data2=np.random.rand(10000, 3)
# 各自加入不同樣本
h1.add_samples(data1)
h2.add_samples(data2)

h1.normalize()
h2.normalize()

# 計算 RMSE
error = rmse(h1, h2)
print("RMSE =", error)

RMSE = 0.00011544321683116725


In [37]:
hist1,_=np.histogramdd(data1,bin_edges)
hist2,_=np.histogramdd(data2,bin_edges)
hist1=hist1/hist1.sum()
hist2=hist2/hist2.sum()
error=np.sqrt(np.mean((hist1 - hist2) ** 2))
print(error)

7.826549866480121e-05


In [38]:
import numpy as np

# 建立隨機樣本
samples = np.random.rand(10000, 3)
bin_edges = [np.linspace(0, 1, 9)] * 3  # 8 bins each

# numpy 官方版本
ref_hist, _ = np.histogramdd(samples, bins=bin_edges)
ref_hist = ref_hist / np.sum(ref_hist)

# 稀疏版本
def sparse_histogramdd(samples, bin_edges):
    bin_idx = [
        np.searchsorted(edges, samples[:, i], side='right') - 1
        for i, edges in enumerate(bin_edges)
    ]
    bin_idx = np.stack(bin_idx, axis=1)
    valid = np.all((bin_idx >= 0) & (bin_idx < [len(e)-1 for e in bin_edges]), axis=1)
    bin_idx = bin_idx[valid]
    hist = {}
    for b in map(tuple, bin_idx):
        hist[b] = hist.get(b, 0) + 1
    total = sum(hist.values())
    for k in hist:
        hist[k] /= total
    return hist

sparse_hist = sparse_histogramdd(samples, bin_edges)

# flatten numpy hist
ref_flat = ref_hist.ravel()

# flatten sparse hist
flat_sparse = np.zeros_like(ref_flat)
shape = ref_hist.shape
for k, v in sparse_hist.items():
    flat_sparse[np.ravel_multi_index(k, shape)] = v

# 比較 RMSE
rmse = np.sqrt(np.mean((ref_flat - flat_sparse) ** 2))
print("RMSE =", rmse)

RMSE = 0.0


In [41]:
a=np.array([[1,2,3],[3,4,5]])

b=a.reshape(3,2)
print(a)
print(b)

[[1 2 3]
 [3 4 5]]
[[1 2]
 [3 3]
 [4 5]]
